In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import re
#import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

**<font size="6">Training Previous Data</font>**

**<font size="5">UPLOAD DATA for Training</font>**

In [2]:
#df = pd.read_csv('VNforModel.csv')
df = pd.read_excel('SG.xlsx').astype(str)
df.head().T

,0,1,2,3,4
channel_id,31,31,31,31,31
brand,breeze,finish,aveeno,farmland,wuwujie
sku_title,GWP Breeze Laundry Capsules with Frag Booster ...,YSL black tube water gloss lip glaze mirror fi...,Aveeno Body Dermexa Emollient Wash For Eczema-...,[BenMart Frozen] Farmland Chicken Leg Cube 2kg...,"WUWUJIE Sock Cleaning Fluid, Deodorizing, Deco..."
channel_sku_id,3542175192-s23369277667,3492616615-s23087462396,290080137-s472933790,3457133073-s22939504264,3372216931-s22839271011
price,100.0,24.7,23.1,12.95,20.0
category,irr,irr,irr,irr,fabric cleaning - hc
sub_category,irr,irr,irr,irr,liquid detergent
mrp,-1.0,80.0,21.95,18.9,10.0
feat_2,36.0,nan,300.0,2000.0,0.3
feat_3,UNIT,nan,ML,GM,L


In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    return text.strip()

In [4]:
df['Title'] = df['sku_title'].apply(clean_text)

In [5]:
res = list(df.columns)
print(res)

['channel_id', 'brand', 'sku_title', 'channel_sku_id', 'price', 'category', 'sub_category', 'mrp', 'feat_2', 'feat_3', 'feat_4', 'feat_10', 'mapping_column', 'country_code', 'page_link', 'Title']


In [6]:
le_category = LabelEncoder()
le_subcategory = LabelEncoder()

In [7]:
df['category_encoded'] = le_category.fit_transform(df['category'])
df['subcategory_encoded'] = le_subcategory.fit_transform(df['sub_category']) 

In [8]:
#from sklearn.preprocessing import LabelEncoder #####################for th error
#le = LabelEncoder() ############################for th error
vectorizer = TfidfVectorizer(ngram_range=(1,2))  # Use unigrams + bigrams
X = vectorizer.fit_transform(df['Title'])

y_category = df['category_encoded']
y_subcategory = df['subcategory_encoded']

X_train, X_test, y_cat_train, y_cat_test = train_test_split(X, y_category, test_size=0.2, random_state=42)
X_train, X_test, y_sub_train, y_sub_test = train_test_split(X, y_subcategory, test_size=0.2, random_state=42)
#y_cat_train = le.fit_transform(y_cat_train)  ############for th error
#y_sub_train = le.fit_transform(y_sub_train)  ###########for th error

In [ ]:
# Train for category
cat_model = XGBClassifier()
cat_model.fit(X_train, y_cat_train)
y_cat_pred = cat_model.predict(X_test)
print("Category Accuracy:", accuracy_score(y_cat_test, y_cat_pred))

# Train for subcategory
sub_model = XGBClassifier()
sub_model.fit(X_train, y_sub_train)
y_sub_pred = sub_model.predict(X_test)
print("Subcategory Accuracy:", accuracy_score(y_sub_test, y_sub_pred))

Category Accuracy: 0.9408326839585884


**<font size="6">Upload DATA and COUNTRY CODE </font>**

In [ ]:
#unmapped_df = pd.read_excel('Tiktokkkkkkkk.xlsx')
unmapped_df = pd.read_excel('any.xlsx').astype(str)
# Add country_code column with fixed value, e.g., 'PH'
unmapped_df['country_code'] = 'VN'
#unmapped_df= pd.read_csv('PH_Shopee_Bidisha.csv', encoding='cp1252')
unmapped_df['title_clean'] = unmapped_df['title'].apply(clean_text)

In [10]:
rename_map = {}

if "brand" in unmapped_df.columns:
    rename_map["brand"] = "brandbycrawl"

if "product_id" in unmapped_df.columns:
    rename_map["product_id"] = "channel_sku_id"

if "asin" in unmapped_df.columns:
    rename_map["asin"] = "channel_sku_id"

unmapped_df = unmapped_df.rename(columns=rename_map)

In [11]:
unmapped_df.head().T

,0,1,2,3,4
channel_sku_id,100003750-s100005192,1001794381-s3650968200,1001810110-s23067853079,1001866831-s22015077713,1001866831-s22015077714
parent asin,100003750,1001794381,1001810110,1001866831,1001866831
title,Pigeon Liquid Cleanser/ 100% Food Grade Bottle...,Colgate Optic White Volcanic Whitening Toothpa...,Kirkland Signature Ultra Clean HE Laundry Dete...,"Fresh HY 4-in-1 Cherry blossom, Lavender , Ros...","Fresh HY 4-in-1 Cherry blossom, Lavender , Ros..."
price,5.65,21.5,62.0,19.5,17.55
mrp,5.65,29.1,62.0,19.5,19.5
sellerName,Baby Kingdom,Colgate,Zenbu Ya,Zenbu Ya,Zenbu Ya
sellerId,3173,1000037704,100048867,100048867,100048867
brandbycrawl,Pigeon,Colgate,Kirkland Signature,Seika,Seika
thumbnail,https://sg-live-01.slatic.net/p/297bcdea42f723...,https://sg-live-01.slatic.net/p/8aad35f8a8436b...,https://filebroker-cdn.lazada.sg/kf/Sf89a70505...,https://filebroker-cdn.lazada.sg/kf/S6df00891d...,https://filebroker-cdn.lazada.sg/kf/S6df00891d...
url,https://www.lazada.sg/products/pdp-i100003750....,https://www.lazada.sg/products/pdp-i1001794381...,https://www.lazada.sg/products/pdp-i1001810110...,https://www.lazada.sg/products/pdp-i1001866831...,https://www.lazada.sg/products/pdp-i1001866831...


In [12]:
X_unmapped = vectorizer.transform(unmapped_df['title_clean'])

y_pred_cat = cat_model.predict(X_unmapped)
y_prob_cat = cat_model.predict_proba(X_unmapped)
threshold = 0.75
max_prob_cat = y_prob_cat.max(axis=1)

y_pred_subcat = sub_model.predict(X_unmapped)
y_prob_subcat = sub_model.predict_proba(X_unmapped)
max_prob_subcat = y_prob_subcat.max(axis=1)

unmapped_df['predicted_category'] = [le_category.inverse_transform([pred])[0] if prob > threshold else "irr" for pred, prob in zip(y_pred_cat, max_prob_cat)]
unmapped_df['predicted_subcategory'] = [le_subcategory.inverse_transform([pred])[0] if prob > threshold else "irr" for pred, prob in zip(y_pred_subcat, max_prob_subcat)]

NameError: name 'cat_model' is not defined

**<font size="6">BRAND MAPPING</font>**

In [6]:
#import re

#import pandas as pd
#unmapped_df = pd.read_excel('TH_OLD_DATA_NEWCAT_combined.xlsx')

In [14]:
unmapped_df.head().T

,0,1,2,3,4
channel_sku_id,1732302988384109654,1733175782840108320,1733306437535827260,1733593556820592398,1731142474860430770
price,219.0,284.05,109.0,109.0,195.01
mrp,219.0,299.0,398.0,398.0,398.0
title,Moisturizing Lightening Lotion Plus 3x Collage...,WHITENING LOTIONS 500ML FROM THAILAND,SIITO Premium Removal - Safe & Natural H...,SIITO Acne Treatments Acne Care Invisible Gel ...,"[QUSTERE Acne Patch] Facial acne patch, colorf..."
url,",https://www.tiktok.com/view/product/173230298...",",https://www.tiktok.com/view/product/173317578...",",https://www.tiktok.com/view/product/173330643...",",https://www.tiktok.com/view/product/173359355...",",https://www.tiktok.com/view/product/173114247..."
thumbnail,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...
feat_10,3P,3P,3P,3P,3P
country_code,PH,PH,PH,PH,PH
title_clean,moisturizing lightening lotion plus 3x collage...,whitening lotions 500ml from thailand,siito premium removal safe natural hea...,siito acne treatments acne care invisible gel ...,qustere acne patch facial acne patch colorful ...
predicted_category,skin - b&w,irr,irr,irr,irr


In [ ]:
brand_list = [
    "A Bonne","abonne", "Aba", "Acecook", "Adabi", "Adidas", "Ajax", "Ajinomoto", "Aji-ngon", "Ajingon", "AHC",
"All Mighty", "Alpecin", "Anessa", "antabax", "Aqua", "Ariel", "Ariel Core", "Ariel Liquids", "Ariel Oxyclean",
"ARM & HAMMER", "ar fum", "ar FÜM", "arFÜM", "Attack", "Attack Kao", "ATP", "Ava", "Aveeno", "Avon",
"Axion", "Axe", "Babi Mild", "Baby Dove", "Babyflo", "Banana Boat", "Bath & Body Works", "Beautilab", "Bega Cheese", "Belo",
"Belo Essentials", "Bench", "Bench Fix Professional", "Best Foods", "Betadine", "Biodem", "Bioderm", "Bioderma", "Bio derma", "Bio-Home",
"bio home", "Bioaqua", "Bioaoua", "Bio-Oil", "Biore", "Block & Glow", "Block & White", "Bold", "Boots", "Born Good",
"Bragg", "Breeze detergent", "Breeze excel", "Breeze liquid", "Breeze usa", "Bremod", "Brushie", "Brut",
"Cafuné ", "Calla", "Carmex",
"Celeteque", "Cera Ve", "Cerave", "Cetaphil", "Champoin", "Champoin Liquids", "Champion", "Charm", "Chante", "Charcoal Clean",
"Chin-Su", "Sunlight", "Cif ", "Citra", "Clarins", "cleanz", "Clean & Clear", "Clear 9", "Clear Antidandruff", "Clear anti-dandruff", "CLEAR Anti Dandruff", "Clear cool",
"Clear hair", "Clear Ice", "Clear men", "Clear mint", "Clear Nose", "CLEAR Sakura", "Clear shampoo", "Clear women", "CLEAR SCALP", "Clorox",
"Close up", "Closeup", "Cocoon", "Colgate", "Colgate Plax", "Comfort afterwash", "Comfort Fabric", "Comfort gentle", "Comfort laundry", "Comfort rinse",
"Comfort thai", "COSRX", "Cream Silk", "CreamSilk", "Crest", "D'Alba", "Daia", "Darlington", "Darlie", "Dell", "Dermorepubliq", 
"Dentiste", "Deo nat", "Deoklear", "Deonat", "Dercos", "Dettol", "Diane", "Dianne", "D-nee", "Dnee",
"Domex", "Double Rich","White Dove", "Dove", "Dove men", "Downy", #baby dove, white dove before dove
"Dr. Clean", "Dr.Clean", "Dr. Kam", "DrKam", "Dr Leo", 
"Dr.Leo", "Dr Pong", "Dr.Pong", "Dr.G", "DR ALTHEA", "DR. ALTHEA", "DR.ALTHEA", "Duo Duo", "DuoDuo", "Dynamo",
"Dyanmo", "Ellips", "Elixir", "elseve", "Enchanteur", "Eskinol", "Estee Lauder", "Eucrin", "Eucerin", "Eversense", "F & B",
"F&B", "Fab", "Fairy", "Familife", "Fineline", "Fine line", "Finish", "Foreo", "Fresh & White", "G2G",
"Gard", "Garnier", "Garnier Light", "Gatsby", "Gelball", "General Mills", "Gillette", "G2G", "Glad2Glow", "Glo", "GMEELAN",
"Good maid", "Good Virtues", "Good Virtues Co", "Goodmaid", "Grace And Glow", "Green Cross", "H & S", "H&S", "Hada Labo", "HadaLabo",
"Hairfix", "Hangy", "Happy", "Hapee", "Hatomugi", "Hazeline", "Head & Shoulders", "Head and shoulder", "Head&Shoulders", "HEXA",
"Hella", "Hellmann", "Hellmann’s", "Hellmanns", "Herbs and spices", "Herbal Essences", "Himalaya Pink Salt", "Himalayan Pink Salt", "Hosoku", "Hygiene", "HygeinC", "HygienC",
"Hygienex","hyperclean", "Human Nature", "Human Nature Baby", "Icare Pharma", "I care Pharma", "Innisfree", "J&J", "JERGENS", "Jergens", "Jiff",
"Jobbie", "Johnson's", "Johnson's Baby", "Johnsons", "Johnsons Baby", "jonhnson baby", "Joins", "JOY", "Joykity", "joseon",
"K-Mom", "K Mom", "KMom", "Ka", "Kaminomoto", "Kanebo", "Kapsa", "Kapsul", "Kate", "Kellogg’s",
"Kérastase", "Kerastase","Kerasys", "Keratin Plus", "KeratinPlus", "Keihl's", "KIEHL", "KIEHL'S", "KIEHLS", "Kirei Kirei",
"Kirkland", "Kleen", "Kleen it", "Kleenit", "Koko Krunch", "Kodomo", "Kormesic", "Knorr", "Koparo", "Kraft Heinz", "Kracie",
"Kundal", "Kutitap", "labn' care", "lab'n care", "labn'care","Lactacyd", "La Roche Posay", "La Roche-Posay", "La Rosche Posay", "La Rosche Possay", "LA ROCHE /POSAY", "La-Roche-Posay",
"Lady Anna", "Lady Choice", "Lady speed", "Lady's Choice", "Lala", "Lakme", "Laneige", "Laundrin", "Lavojoy", "Lee Kum Kee",
"Lenor", "Lexi", "Léxi", "Lifebuoy", "Lifebouy", "Listerine", "Listrine", "Lix", "L'Occitane", "Lock and lock", "Lonkey", 
"Loreal Absolut", "Loreal Inforcer", "Loreal Liss",
"Loreal Professional", "LOreal Professionnel", "Loreal Pro", "Loreal Sensi", "Loreal Silver", "Loreal Vitamino",
"Loreal Professionnel", "L’Oreal Professional", "loreal professional",
"Loreal Vitamino", "Loreal Vitamino", "Loreal Absolut", "Loreal Silver", "Loreal Liss", "Loreal Inforcer", "Loreal Pro", "Loreal Sensi",
"L'Oreal Vitamino", "L'Oreal Vitamino", "L'Oreal Absolut", "L'Oreal Silver", "L'Oreal Liss", "L'Oreal Inforcer", "L'Oreal Pro", "L'Oreal Sensi",
"L'Oréal Vitamino", "L'Oréal Vitamino", "L'Oréal Absolut", "L'Oréal Silver", "L'Oréal Liss", "L'Oréal Inforcer", "L'Oréal Pro", "L'Oréal Sensi"
"Loreal", "L'Oreal", "L'Oréal", "L'Oreal Paris", "L'Oréal Paris", #loreal paris after professionnel keywords
"Deparis","De paris", "Paris", #"Deparis","De paris", "Paris" after loreal paris
"Love Beauty & Planet", "Love Beauty and Planet", "Lowe", "Luodais", "Lux",
"Luxe Organix", "Luxe", "LYDIMOON", "Lysol", #Luxe Organix before lux
"Maggi", "Magnolia", "Mama earth", "mama earth", "mamaearth", "Mama Lemon", "Mama Sita’s",
"Mamee Chef", "Mame Chef", "Mane'n Tail", "Manen Tail", "Marvis", "Master", "Maybelline", "Maxa", "Maxipeel", "Maxkleen", "medicube",
"Mentholatum", "Method", "Michiru", "Mighty Clean", "Mightly Clean", "Milcu", "Mielle", "Milo", "Mise En Scene","Mise En", "Mistine",
"Mizumi", "Morocconoil", "Mr Muscle", "Mr. Muscle", "Mr.Muscle", "Mr.Fresh", "Ms. Tsung", "My Hao", "Myra", "Nam Ngư",
"Nard", "Nars", "Naturista", "Nature's Miracle", "Nestlé", "Nestle", "Nestum", "Nerman", "Neutrogena", "Nivea",
"Nivea men", "Nivea Sun", "Nizoral", "Nike", "Nguyên Vương", "OGX", "ogx", "Olay", "Olay men", "Old Spice",
"Omo","on1", "ON THE BODY", "ON: THE BODY", "On: The Body", "Original Sprout", "Oriental Beauty", "Oriental Princess", "Oral B", "Oral-B", "P/O Care",
"P.O. Care", "PO Care", "P/S", "PAPA FEEL", "Palmolive", "Palmolive Men", "Palmer's", "Panasonic", "Pantene", "Paris",
"Pepsodent", "Persil", "Perspirex", "Philips", "Philips Sonicare", "Pigeon", "Playboy", "Pond", "Pond's", "Pond’s",
"Ponds", "Power Root", "Pril", "Propolinse", "Protex", "Pure n'Fresh Baby", "Purite","Purité", "Rdl", "Refre", "Rejoice",
"Revitalift", "Rexona", "Rexona female", "rexona male", "Rhys Man", "Romano", "ROUND LAB", "Ryo", "Sadoer", "Safeguard",
"Safe guard", "Sao thai duong", "Sao thái dương", "sanzoft", "Schwarzkopf", "Scrubbing Bubbles", "Secret", "Senka", "Sensodyne", "Selsun", "Seaways",
"seaway", "Shield", "Shawill", "SHE Eureka", "SHEEureka", "Shiseido", "Shokubutsu", "Simple", "Silka", "SilkaSoap",
"Skin 1004", "Skin Aqua", "Skin1004", "SKINEVER", "Skintific", "Skippy", "Smoothe", "Smooth E", "Snow Clear", "Soap & Glory",
"So Royale", "SoRoyale", "Softlan", "Some By Mi", "Speed stick", "St. Ives", "ST IVES", "St.Ives", "Summerie",
"Sun Bum", "Sunlight", "Sunplay", "Sunsilk", "Sunslik", "Surf", "Surf ", "Systema", "Tender Care", "TenderCare",
"The Body Shop", "The Bodyshop", "The Ordinary", "Tiger", "Tide", "Tide Liquids", "Tide Perfect Clean", "TopGia", "top gia", "Tresemm",
"Tresemme", "TRESemmé", "Tros", "Tsubaki", "UFC", "Uic", "Uni Love", "UniLove", "UniKeratin", "Unove",
"Usmile ", "Vanish", "Vase line", "Vaseline", "Veemil", "Verite", "Vichy", "Vigor", "VIKADA", "Vim",
"Vita keratin", "Vitakeratin", "Walch", "Watnose", "Watsons", "watson", "Well hair", "Wellhair", "Wings",
"Wing ", "Wings Liquids", "WUWUJIE", "Xiaomi", "X-Men", "Xmen", "Yan Wal Yun", "yen yen", "yenyen", "Zenutrients",
"Zip", "Zonrox", "Alpha Arbutin", "Kao" #Alpha Arbutin, kao should be in the last
]


In [ ]:
# Brands that should match only at the beginning or as full words
strict_start_brands = [
    "&Honey", "Tide", "Top", "Lux", "Wings", "Joy", "Usmile", "Joykity", "Top powder","omo", "Axe", "Fab", "Joy ", "ka", "comfort", "Glo",
    "Aba", "AHC", "Adabi", "Adidas", "Ajinomoto", "Dermorepubliq", "Alpha Arbutin", "Ariel", "Aveeno", "ATP", "Avon", "All Mighty", "Babi Mild",
    "Baby Dove", "Babyflo", "Banana Boat", "Bath & Body Works", "Beautilab", "Bega Cheese", "Belo ", "Breeze",
    "Bench", "Bench Fix Professional", "Betadine", "Biore", "Block & Glow", "Bold", "Boots", "Bremod",
    "Brut", "Carmex", "Celeteque", "Cera Ve", "Cerave", "Cetaphil", "Champoin", "Champion", "Charcoal Clean", "Cif ", "CLEARNOSE", "CLEAR NOSE",
    "Citra", "Clarins", "Clean & Clear", "Clear men", "Clear shampoo", "Clorox", "Close up", "Closeup", "Colgate", "Comfort",
    "Colgate Plax", "Cream Silk", "CreamSilk", "Crest", "Daia", "Darlie", "Deoklear", "Deonat", "Dettol", "Dentiste", "Del",
    "Diane", "Double Rich", "Downy", "DrKam", "Dr Leo", "Dr.Leo", "Dr Pong", "Dr.Pong", "Dr. Clean", "Dr.Clean", "Dr. ville", "Dr.ville",
    "DuoDuo", "Dynamo", "Dyanmo", "Ellips", "Enchanteur", "Eskinol", "Estee Lauder", "Eucerin", "Eversense", 
    "Familife", "Fairywill", "F&B Food Solutions", "Fineline", "Foreo", "Fresh & White",
    "Gard", "Garnier", "Garnier Light", "Gatsby", "Gelball",  "GMEELAN", "Good Virtues Co.", "Goodmaid", "Good maid",
    "Grace And Glow", "Green Cross", "Hada Labo", "HadaLabo", "Hairfix", "Hangy", "Hapee", "Head & Shoulders", "Head&Shoulders", "Head & Shoulder",
    "Hella", "Hellmann’s", "Hellmanns", "HEXA Food", "Horlicks", "Human Nature", "Human Nature Baby", "HygienC",
    "Hygiene", "Hygienex", "Himalayan Pinksalt", "Innisfree", "Jergens", "Jobbie", "Johnson's", "Johnsons", "Johnson's Baby", "Johnsons Baby", "Joins",
    "Lavojoy", "Joykity", "Kerastase", "Kin Gingival", "Kirkland", "Kleenit", "Kleen it", "Kleen", "Knorr", "Kraft Heinz",
    "Kundal", "Kutitap", "La Roche-Posay","La Roche Posay", "Lady's Choice", "Ladys Choice", "Lakme", "Laneige", "Lee Kum Kee",
    "Listerine", "L'Occitane", "Lock and lock", "Lonkey","L'Oréal","Loreal", "L'Oreal Paris", "L'Oréal Paris", "Deparis","De paris", "Paris",
    "Love Beauty & Planet","Luxe ", "Lux ", "Lifebuoy", "Lifebouy", "Mama Lemon", "Mamee Chef", "Mane'n Tail", "Manen Tail", "Maggi",
    "Marvis", "Master", "Maxkleen", "Mentholatum", "Milcu", "Mielle", "Mighty Clean", "Milo",
    "Mise En Scene Perfecting Serum", "Mistine", "Mizumi", "Mr Muscle", "Mr.Muscle", "Mr.Fresh", "Ms. Tsung", "Nard",
    "Nars", "Nature's Miracle", "Naturista", "Nestlé", "Nestle", "Nestum", "Neutrogena", "Nike", "Nivea Sun","Nivea",
    "Olay", "Old Spice", "Omo ", "Oral B", "Oral-B", "Original Sprout", "Oriental Princess", "Oriental Beauty","On: The Body","On The Body", "PAPA FEEL", "Palmolive",
    "Palmer's", "Panasonic", "Pantene", "Pepsodent", "Persil", "Perspirex", "Philips Sonicare",
    "Physiogel", "Playboy", "PO Care","P.O. Care", "Pond's", "Ponds", "Power Root", "Pride", "Propolinse", "Protex",
    "Pure n'Fresh Baby", "Rexona", "Rejoice", "Rhys Man", "Rdl ", "RtopR", "Romano", "Sadoer", "Safeguard", "Safe guard", "Saskin", "Sas skin",
    "Schwarzkopf", "Scrubbing Bubbles", "Secret", "Seaways", "Sensodyne", "Selsun", "Shawill", "SHEEureka","Silka",
    "Shiseido", "Shokubutsu", "Simple", "Skintific", "Skippy", "Smoothe", "Soap & Glory",
    "Softlan", "Some By Mi","Speed ", "St. Ives", "St.Ives", "Summerie", "Sunlight", "Sunplay", "Sunsilk", "Sunslik", "Surf", "Systema",
    "Tender Care", "The Body Shop", "The Ordinary", "top gia", "Tresemme", "TRESemmé", "Tresemm",
    "Tros", "Tsubaki", "Uni Love", "UniLove", "Usmile ", "Veemil", "Verite", "Vichy", "Vigor", "Vaseline", "Vase line", "Vitakeratin", 
    "Walch", "Watnose", "Watsons", "Well hair","Wellhair", " Wing ", "Zenutrients", "Zip", "Zonrox", "Xiaomi",
    "ar FÜM", "arFÜM", "ar fum"
]


# === Clean function ===
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)  # remove punctuation
    return text.strip()

# === Clean brand lists and create lookup dicts ===
brand_dict = {clean_text(b): b for b in brand_list}  # normalized -> original
strict_start_dict = {clean_text(b): b for b in strict_start_brands}

# Precompute brand word lists
brand_words_dict = {b: b.split() for b in brand_dict.keys()}
strict_start_words_dict = {b: b.split() for b in strict_start_dict.keys()}

def find_brand_fast(title):
    """
    Optimized brand matching: exact word match, multi-word brands, strict start priority.
    """
    title_clean = clean_text(title)
    title_words = title_clean.split()
    n_words = len(title_words)

    # --- Priority 1: strict_start_brands at start ---
    for brand_norm, brand_words in strict_start_words_dict.items():
        if title_words[:len(brand_words)] == brand_words:
            return strict_start_dict[brand_norm]  # return original formatting

    # --- Priority 2: other brands anywhere ---
    # Use sliding window approach
    for brand_norm, brand_words in brand_words_dict.items():
        len_bw = len(brand_words)
        if len_bw > n_words:
            continue
        for i in range(n_words - len_bw + 1):
            if title_words[i:i+len_bw] == brand_words:
                return brand_dict[brand_norm]

    return None

# Apply to your DataFrame
unmapped_df['brandbyModel'] = unmapped_df['title_clean'].apply(find_brand_fast)

In [ ]:
# Define mapping in grouped format
brand_groups = {
    "a bonne": ["abonne"],
    "ar fum": ["ar FÜM", "arFÜM", "ar fum"],
    "amway": ["g&h"],
    "Best Foods":["Best Foods", "BestFoods"],
    "Block & Glow": ["Block & White","Block & Glow"],
    "Bioaqua": ["Bioaqua", "Bioaoua"],
    "Breeze": ["Breeze excel", "Breeze usa", "Breeze detergent", "Breeze liquid"],
    "Champion": ["Champoin", "Champion"],
    "Clear": ["Clear men", "Clear women", "Clear shampoo", "Clear mint", "Clear 9", "Clear cool", "Clear anti-dandruff", "Clear Ice",
    "CLEAR SCALP", "Clear hair", "CLEAR Sakura", "Clear Antidandruff", "CLEAR Anti Dandruff"],
    "Closeup": ["Closeup", "Close up"],
    "comfort": ["Comfort Fabric", "Comfort thai", "Comfort gentle", "Comfort rinse", "Comfort afterwash", "Comfort laundry"],
    "cosway": ["powermax"],
    "Cream Silk": [ "Cream Silk", "CreamSilk" ],
    "Deonat": ["Deonat", "Deo nat"],
    "D-nee": ["D-nee", "Dnee"],
    'Dr.Pong': ["Dr Pong", "Dr.Pong"],
    'Dr.Clean': ["Dr. Clean", "Dr.Clean"],
    'Dr.Ville': ["Dr. ville", "Dr.ville"],
    'Dynamo': ["Dynamo", "Dyanmo"],
    'Fineline': ["Fineline", "Fine line", "Finelime"],
    'Glad2Glow': ["G2G", "Glad2Glow"],
    'Goodmaid': ["Goodmaid", "Good maid"],
    'Good Virtues Co.': ["Good Virtues Co", "Good Virtues"],
    'Hada Labo': ["Hada Labo", "HadaLabo"],
    'Head & Shoulders': ["Head & Shoulders", "Head&Shoulders" , "Head & Shoulder","Head and shoulder","H&S","H & S"],
    'Hellmann’s': ["Hellmann’s", "Hellmann", "Hellmanns"],
    'Human Nature': ["Human Nature", "Human Nature Baby"],
    "Hellmann's": ["Hellmann’s", "Hellmanns"],
    "Himalaya Pink Salt": ["Himalaya Pink Salt","Himalayan Pink Salt", "Himalayan Pinksalt"],
    "Beauty of Joseon": ["joseon"],
    "Johnson's": ["Johnson's", "Johnsons", "Johnson's Baby", "Johnsons Baby", "JOHNSON&JOHNSON", "johnson baby", "J-ohnsos Baby", "J&J"],
    "Kerastase": ["Kérastase", "Kerastase"],
    "Keratin Plus": ["Keratin Plus", "KeratinPlus"],
    "K-Mom": ["K-Mom", "KMom", "K Mom"],
    "Kleenit": ["Kleenit", "Kleen it"],
    "KIEHL'S": ["KIEHL'S", "KIEHLS", "KIEHL", "Keihl's"],
    "labn'care": [ "labn' care", "labn'care", "comfy"],
    "Lady's Choice": ["Lady's Choice", "Ladys Choice", "Lady Choice"],
    "Lexi": ["Léxi", "Lexi"],
    "Loreal Paris": ["L'Oreal", "L'Oréal", "Loreal", "L'Oreal Paris", "L'Oréal Paris", "Revitalift", "elseve"],
    
    "Loreal Professionnel": ["Loreal Professionnel", "Professionnel", "L’Oreal Professional", "loreal professional",
    "Loreal Vitamino", "Loreal Vitamino", "Loreal Absolut", "Loreal Silver", "Loreal Liss", "Loreal Inforcer", "Loreal Pro", "Loreal Sensi",
    "L'Oreal Vitamino", "L'Oreal Vitamino", "L'Oreal Absolut", "L'Oreal Silver", "L'Oreal Liss", "L'Oreal Inforcer", "L'Oreal Pro", "L'Oreal Sensi",
    "L'Oréal Vitamino", "L'Oréal Vitamino", "L'Oréal Absolut", "L'Oréal Silver", "L'Oréal Liss", "L'Oréal Inforcer", "L'Oréal Pro", "L'Oréal Sensi"],
    
    "La Roche-Posay": ["La-Roche-Posay", "La Roche-Posay","La Roche Posay", "LA ROCHE /POSAY"],
    "Lifebuoy": ["Lifebuoy", "Lifebouy"],
    "Luodais": ["LUÓDAIS", "L'UODAIS", "Luodais"],
    "Mane'n Tail": ["Mane'n Tail", "Manen Tail"],
    "Mise En Scene": ["Mise En Scene", "Mise En"],
    "Nestle": ["Nestlé", "Nestle", "Nestum", "Milo"],
    "Nivea": ["Nivea Sun","Nivea"],
    "Mr Muscle": ["Mr Muscle", "Mr.Muscle","Mr. Muscle"],
    "Nguyen Vuong": ["Nguyen Vuong"],
    "Oral-B": ["Oral B", "Oral-B"],
    "Oriental Princess": ["Oriental Princess", "Oriental Beauty"],
    "On The Body": ["On: The Body", "On The Body", "On: The Body"],
    "PO Care": ["PO Care", "P.O. Care"],
    "Pond's": ["Pond's", "Ponds", "Pond"],
    "Sabai Fluocaril": ["Fluocaril"],
    "Safeguard": ["Safeguard", "Safe guard"],
    "SHEEureka": ["SHEEureka", "SHE Eureka"],
    "Sof & Mmmmmm":["Sof &"],
    "So Royale": ["So Royale", "SoRoyale"],
    "Saskin": ["Saskin", "Sas skin"],
    "SKIN1004": ["SKIN1004", "SKIN 1004"],
    "Speed stick":["Speed stick", "Lady speed"],
    "St.Ives": ["St. Ives", "St.Ives", "ST IVES"],
    "Sunsilk": ["Sunsilk", "Sunslik"],
    "Thai Duong": ["sao thai duong",  "sao thái dương" ,"thai duong"],
    "Tresemme": ["Tresemme", "TRESemmé", "Tresemm"],
    "Uni Love": ["Uni Love", "UniLove"],
    "Vitakeratin": ["Vitakeratin", "Vita keratin"],
    "Watsons": ["Watsons", "watson"],
    "Vaseline": ["Vaseline", "Vase line"],
    "Wellhair": ["Wellhair", "Well hair"],
    "X-Men": ["X-Men","Xmen"],
    "yen yen": [ "yenyen", "yen yen"]  
}

# Flatten to a reverse mapping
brand_map = {alias: brand for brand, aliases in brand_groups.items() for alias in aliases}

# Apply mapping
unmapped_df['brandbyModel'] = unmapped_df['brandbyModel'].map(brand_map).fillna(unmapped_df['brandbyModel'])

In [18]:
brand_rules = [
    ("luxe", r"\borganix\b", "Luxe Organix"),
    ("shiseido", r"\bfino\b", "Fino"),
    ("shiseido", r"\btsubaki\b", "Tsubaki"),
    ("elixir", r"\bkerastase\b", "kerastase"),
]

for parent, keyword, final_brand in brand_rules:
    mask = (
        unmapped_df['brandbyModel'].fillna("").str.strip().str.lower() == parent
    ) & (
        unmapped_df['title_clean'].str.contains(keyword, case=False, na=False)
    )
    unmapped_df.loc[mask, 'brandbyModel'] = final_brand


In [19]:
unmapped_df.head(50).T

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
channel_sku_id,1732302988384109654,1733175782840108320,1733306437535827260,1733593556820592398,1731142474860430770,1733295404466407017,1733091627174627308,1730308948275988537,1731911475202393214,1731013029846289249,...,1729668653079103966,1730829738363882301,1729669231404091870,1731074411612441557,1729816375965617067,1729633508483631985,1730139736875698307,1731046429367896824,1730139733839874179,1730856296398291975
price,219.0,284.05,109.0,109.0,195.01,119.0,139.0,69.0,384.12,35.0,...,49.0,199.0,59.0,139.0,45.0,43.0,77.22,26.0,77.22,182.6
mrp,219.0,299.0,398.0,398.0,398.0,398.0,339.0,159.0,776.0,35.0,...,49.0,199.0,59.0,149.0,45.0,43.0,78.0,26.0,78.0,182.6
title,Moisturizing Lightening Lotion Plus 3x Collage...,WHITENING LOTIONS 500ML FROM THAILAND,SIITO Premium Removal - Safe & Natural H...,SIITO Acne Treatments Acne Care Invisible Gel ...,"[QUSTERE Acne Patch] Facial acne patch, colorf...",SIITO Acne Care Invisible Gel is suitable for ...,[NEW LAUNCH] Y.O.U AcnePlus BHA Acne Patch | V...,BUY 1 TAKE 2 YESHUI 40g Green Tea Mud Mask Bla...,【Limited-time offer】QUSTERE facial large hydro...,Mugwort Acne Clay Mask Stick Gentle Cleansing ...,...,Alyssa Luis Adobo Mani NUTS Pack Protein Snack...,Adobo Flakes Classic 220ml,Alyssa Luis MIXED NUTS Pack Protein Snack,MIX Crispy Snacks Peanuts Cornick Popbeans 400g,Mama Sita's Meal Mix Spicy Caldereta 50g,"YM’s Cashew Nuts, Peanut Brittle, Sweet Beans ...",Dingdong Snack Mix Mixed Nuts 95G x 3PCs,"Datu Puti Soy Sauce 350ml - AllMart Grocery, S...",Dingdong Mixed Nuts Original 100G x 3Pcs,"555 Tuna Adobo 155g pack of 6 - Healthy, High ..."
url,",https://www.tiktok.com/view/product/173230298...",",https://www.tiktok.com/view/product/173317578...",",https://www.tiktok.com/view/product/173330643...",",https://www.tiktok.com/view/product/173359355...",",https://www.tiktok.com/view/product/173114247...",",https://www.tiktok.com/view/product/173329540...",",https://www.tiktok.com/view/product/173309162...",",https://www.tiktok.com/view/product/173030894...",",https://www.tiktok.com/view/product/173191147...",",https://www.tiktok.com/view/product/173101302...",...,",https://www.tiktok.com/view/product/172966865...",",https://www.tiktok.com/view/product/173082973...",",https://www.tiktok.com/view/product/172966923...",",https://www.tiktok.com/view/product/173107441...",",https://www.tiktok.com/view/product/172981637...",",https://www.tiktok.com/view/product/172963350...",",https://www.tiktok.com/view/product/173013973...",",https://www.tiktok.com/view/product/173104642...",",https://www.tiktok.com/view/product/173013973...",",https://www.tiktok.com/view/product/173085629..."
thumbnail,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...
feat_10,3P,3P,3P,3P,3P,3P,3P,3P,3P,3P,...,3P,3P,3P,3P,3P,3P,3P,3P,3P,3P
country_code,PH,PH,PH,PH,PH,PH,PH,PH,PH,PH,...,PH,PH,PH,PH,PH,PH,PH,PH,PH,PH
title_clean,moisturizing lightening lotion plus 3x collage...,whitening lotions 500ml from thailand,siito premium removal safe na

**<font size="6">FIX THUMBNAIL LINK</font>**

In [20]:
import ast
import pandas as pd

# Determine which column to use
source_col = None
for col in ['thumbnail', 'media']:
    if col in unmapped_df.columns:
        source_col = col
        break

def get_first_media_link(media):
    if not media or pd.isna(media):
        return None
    # If it's a stringified list
    if isinstance(media, str) and media.startswith("[") and media.endswith("]"):
        try:
            media_list = ast.literal_eval(media)
        except Exception:
            return None
    # If it's already a list
    elif isinstance(media, list):
        media_list = media
    # If it's a single URL string
    elif isinstance(media, str):
        return media.strip()
    else:
        return None
    
    # Return first link if exists
    return media_list[0] if media_list else None

if source_col:
    unmapped_df['thumbnail'] = unmapped_df[source_col].apply(get_first_media_link)
#else:
    # If neither column exists, create an empty thumbnail column
    #unmapped_df['thumbnail'] = None


**<font size="6">PACK SIZE, PACK TYPE</font>**

In [1]:
import re

import pandas as pd
#unmapped_df = pd.read_excel('SG_problem_skus.xlsx').astype(str)

In [ ]:
import re
import pandas as pd

def extract_pack_size_and_type(title):
    if not isinstance(title, str):
        return None, None

    text = title.lower()
    text = text.replace(",", "").replace("×", "x").replace("&", "and")

    # --- remove promotions like "buy 1 get 2 free", "b1g1", "buy 2 get 3"
    text = re.sub(r"buy\s*\d+\s*get\s*\d+(\s*free)?", "", text)
    text = re.sub(r"\bb\s*\d+\s*g\s*\d+\b", "", text)

      # --- remove load / laundry / wash counts (CRITICAL FIX)
    text = re.sub(
        r'\b\d+\s*(load|loads|laundry|lifebuoy)\b',
        '',
        text
    )

    # --- remove brand-like tokens with letters+numbers together (e.g. glad2glow, g2g, b2b)
    text = re.sub(r"\b[a-z]+\d+[a-z]+\b", "", text)

    # --- remove multipack mentions not sizes (3pcs, 10 pieces, etc.)
    text = re.sub(r"\b\d+\s*(pcs|pieces|piece|pack|bundle)\b", "", text)

    # --- unit normalization map
    unit_map = {
        'g': 'g', 'gm': 'g', 'gr': 'g', 'grams': 'g',
        'kg': 'kg', 'kgs': 'kg', 'kilo': 'kg',
        'ml': 'ml',
        'l': 'l', 'liter': 'l', 'liters': 'l', 'litre': 'l',
        'gallon': 'gallon'
    }

    # --- unified unit conversion
    def normalize_size(size, unit):
        if unit == "kg":
            return size * 1000, "g"
        elif unit == "l":
            return size * 1000, "ml"
        elif unit == "gallon":
            return size * 3785, "ml"   # US gallon
        return size, unit

    # --------------------------------------------------
    # 1. Range pattern "10-16l"
    # --------------------------------------------------
    match = re.search(
        r'(\d+(?:\.\d+)?)[-–](\d+(?:\.\d+)?)(\s*ml|\s*g|\s*gm|\s*gr|\s*grams|\s*kg|\s*l|\s*gallon|\s*liter|\s*liters|\s*litre|\s*kilo)',
        text
    )
    if match:
        size = min(float(match.group(1)), float(match.group(2)))
        unit = unit_map.get(match.group(3).strip(), match.group(3).strip())
        size, unit = normalize_size(size, unit)
        return size, unit

    # --------------------------------------------------
    # 2. Slash-separated values "1kg/500g/250g"
    # --------------------------------------------------
    match = re.findall(r'(\d+(?:\.\d+)?)(kg|g|gm|gr|grams|ml|l|gallon)', text)
    if "/" in text and match:
        values = []
        final_unit = None
        for val, unit in match:
            size = float(val)
            unit = unit_map.get(unit.strip(), unit.strip())
            size, unit = normalize_size(size, unit)
            values.append(size)
            final_unit = unit
        return min(values), final_unit

    # --------------------------------------------------
    # 3. Standard matches
    # --------------------------------------------------
    match = re.findall(
        r'(\d+(?:\.\d+)?)(\s*ml|\s*g|\s*gm|\s*gr|\s*grams|\s*kg|\s*kgs|\s*l|\s*gallon|\s*liter|\s*liters|\s*litre|\s*kilo)',
        text
    )
    if match:
        sizes = []
        final_unit = None
        for val, unit in match:
            size = float(val)
            unit = unit_map.get(unit.strip(), unit.strip())
            size, unit = normalize_size(size, unit)
            sizes.append(size)
            final_unit = unit

        if sizes:
            if ("+" in text) or (" and " in text):
                return sum(sizes), final_unit
            else:
                return min(sizes), final_unit

    return None, None


In [23]:
#usage on DataFrame 
unmapped_df[['pack_size_predicted', 'pack_type_predicted']] = unmapped_df['title'].apply(lambda x: pd.Series(extract_pack_size_and_type(x)))

In [24]:
unmapped_df.head(50).T

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
channel_sku_id,1732302988384109654,1733175782840108320,1733306437535827260,1733593556820592398,1731142474860430770,1733295404466407017,1733091627174627308,1730308948275988537,1731911475202393214,1731013029846289249,...,1729668653079103966,1730829738363882301,1729669231404091870,1731074411612441557,1729816375965617067,1729633508483631985,1730139736875698307,1731046429367896824,1730139733839874179,1730856296398291975
price,219.0,284.05,109.0,109.0,195.01,119.0,139.0,69.0,384.12,35.0,...,49.0,199.0,59.0,139.0,45.0,43.0,77.22,26.0,77.22,182.6
mrp,219.0,299.0,398.0,398.0,398.0,398.0,339.0,159.0,776.0,35.0,...,49.0,199.0,59.0,149.0,45.0,43.0,78.0,26.0,78.0,182.6
title,Moisturizing Lightening Lotion Plus 3x Collage...,WHITENING LOTIONS 500ML FROM THAILAND,SIITO Premium Removal - Safe & Natural H...,SIITO Acne Treatments Acne Care Invisible Gel ...,"[QUSTERE Acne Patch] Facial acne patch, colorf...",SIITO Acne Care Invisible Gel is suitable for ...,[NEW LAUNCH] Y.O.U AcnePlus BHA Acne Patch | V...,BUY 1 TAKE 2 YESHUI 40g Green Tea Mud Mask Bla...,【Limited-time offer】QUSTERE facial large hydro...,Mugwort Acne Clay Mask Stick Gentle Cleansing ...,...,Alyssa Luis Adobo Mani NUTS Pack Protein Snack...,Adobo Flakes Classic 220ml,Alyssa Luis MIXED NUTS Pack Protein Snack,MIX Crispy Snacks Peanuts Cornick Popbeans 400g,Mama Sita's Meal Mix Spicy Caldereta 50g,"YM’s Cashew Nuts, Peanut Brittle, Sweet Beans ...",Dingdong Snack Mix Mixed Nuts 95G x 3PCs,"Datu Puti Soy Sauce 350ml - AllMart Grocery, S...",Dingdong Mixed Nuts Original 100G x 3Pcs,"555 Tuna Adobo 155g pack of 6 - Healthy, High ..."
url,",https://www.tiktok.com/view/product/173230298...",",https://www.tiktok.com/view/product/173317578...",",https://www.tiktok.com/view/product/173330643...",",https://www.tiktok.com/view/product/173359355...",",https://www.tiktok.com/view/product/173114247...",",https://www.tiktok.com/view/product/173329540...",",https://www.tiktok.com/view/product/173309162...",",https://www.tiktok.com/view/product/173030894...",",https://www.tiktok.com/view/product/173191147...",",https://www.tiktok.com/view/product/173101302...",...,",https://www.tiktok.com/view/product/172966865...",",https://www.tiktok.com/view/product/173082973...",",https://www.tiktok.com/view/product/172966923...",",https://www.tiktok.com/view/product/173107441...",",https://www.tiktok.com/view/product/172981637...",",https://www.tiktok.com/view/product/172963350...",",https://www.tiktok.com/view/product/173013973...",",https://www.tiktok.com/view/product/173104642...",",https://www.tiktok.com/view/product/173013973...",",https://www.tiktok.com/view/product/173085629..."
thumbnail,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...
feat_10,3P,3P,3P,3P,3P,3P,3P,3P,3P,3P,...,3P,3P,3P,3P,3P,3P,3P,3P,3P,3P
country_code,PH,PH,PH,PH,PH,PH,PH,PH,PH,PH,...,PH,PH,PH,PH,PH,PH,PH,PH,PH,PH
title_clean,moisturizing lightening lotion plus 3x collage...,whitening lotions 500ml from thailand,siito premium removal safe na

In [109]:
#download data
#unmapped_df.to_excel('VN_OLD_NORMALIZE_PACKTYPE.xlsx', index=False, engine='openpyxl')

In [ ]:
import re

import pandas as pd
#upload data
#unmapped_df = pd.read_excel('VN.xlsx').astype(str)
#unmapped_df.head().T

In [64]:
#unmapped_df.drop(columns=['uni+comp1'], inplace=True)

**<font size="5">MARK NEW UNI+COMP1 BRAND</font>**

In [25]:
VN_unilever_brands = ["Surf", "Omo", "Comfort", "Dove", "Sunsilk", "Tresemme", "Lux", "Clear", "Axe", "Lifebuoy", "P/S", "Closeup", "Vaseline", "Pond's", "Hazeline", "Simple", "AHC", "Block & Glow", "Knorr", "Lady's Choice", "Best Foods", "Sunlight", "Vim", "Rexona"]
VN_unilever_comp1_brands = ["Ariel", "Chante", "Vanish", "Lix", "Tiger", "Downy", "Pantene", "Head & Shoulders", "Loreal Paris", "Selsun", "Icare Pharma", "Lexi", "Oral-B", "Colgate", "Nivea", "CeraVe", "Olay", "Cetaphil", "La Roche-Posay", "Eucrin", "Anessa", "Garnier", "Dettol", "The Body Shop", "Palmolive", "My Hao", "Finish", "Net", "Chin-Su", "Nam Ngu", "Thuan Phat", "Lien Thanh", "Khai Hoan", "Ajinomoto", "Aji-ngon", "Masan", "Nestle", "Maggi", "Acecook", "Enchanteur", "Romano", "X-Men", "Nerman"]

MY_unilever_brands = [
    "Ahc", "Ajinomoto", "Aveeno", "Axe", "Best Foods", "Block & Glow", "Breeze",
    "Cerave", "Cetaphil", "Cif", "Clear", "Comfort", "Dettol", "Domex", "Downy",
    "Dr. Pong", "Dove", "Dynamo", "Eucerin", "Hazeline", "Heinz", "Hellmann’s",
    "Horlicks", "Kewpie", "Knorr", "Lady Anna", "Lady's Choice", "Lifebuoy",
    "Lipton", "Lonkey", "Lux", "Magnolia", "Mise En Scene", "Neutrogena",
    "Nivea", "Pantene", "Persil", "Pond's", "Rexona", "Safeguard", "Seaways",
    "Selsun", "Simple", "Skintific", "Sunlight", "Sunsilk", "TRESemme",
    "UFC", "Vaseline", "Zip"
]
MY_unilever_comp1_brands = [
    "Aji-ngon", "Anessa", "Blistex", "Cerave", "Chin-Su", "Clear & Clear",
    "Daia", "Darlington", "Dettol", "Downy", "Garnier", "glo",
    "Good Virtues Co", "Jobbie", "Jonhnson's", "Kao", "Lily’s", "Loreal Paris","Loreal Professionnel",
    "Lonkey", "Maggi", "Mamee Chef", "Masan", "Mr. Muscle", "Nestle",
    "Nivea", "Old Spice", "Palmolive", "Pantene", "rejoice", "Scrubbing Bubbles",
    "Seaways", "Selsun", "Skippy", "Softlan", "Watsons", "Zip"
]

PH_unilever_brands = ["Rexona", "Dove", "Axe", "Breeze", "Surf", "Cream Silk", "Sunsilk", "Tresemme", "Vitakeratin", "Clear", "Vaseline", "Pond's", "Eskinol", "Lifebuoy", "Block & Glow", "Love Beauty & Planet", "Baby Dove", "Lady's Choice", "Best Foods", "Knorr"]

PH_unilever_comp1_brands = ["Nivea", "Pantene", "Head & Shoulders", "Keratin Plus", "Olay", "Silka", "RDL", "Safeguard", "Johnson's", "Cetaphil", "Ariel", "Champion", "Calla", "Wings", "Maxkleen", "Downy", "Kewpie", "Heinz", "Magnolia", "UFC", "Maggi", "Ajinomoto", "Mama Sita’s"]

SG_unilever_brands = [
    "ahc", "Ajinomoto", "Best Foods", "Breeze", "Cetaphil", "Clear",
    "Comfort", "Dove", "Dynamo", "Finish", "hazeline", "Kerastase",
    "knorr", "La Roche-Posay", "Lady's Choice", "Lakme", "Lifebuoy",
    "Lix", "Loreal Paris", "Loreal Professionnel", "Lux", "My Hao",
    "Net", "nivea", "Persil", "Ponds", "pond's", "Rexona", "Shokubutsu",
    "Simple", "simple", "sunlight", "Sunsilk", "TRESemme", "Tsubaki",
    "Vaseline", "vim"
]

SG_unilever_comp1_brands = [
    "Aji-ngon", "Anessa", "Attack", "Attack Kao", "Biore", "Cera Ve",
    "Chin-Su", "D'Alba", "Diane", "Downy", "Dr. ville", "Duck",
    "Fairy", "Gatsby", "Kao", "Kao Attack", "Laneige", "L'Occitane",
    "Maggi", "Masan", "Maxa", "Mentholatum", "Nars", "Nestle",
    "Pantene", "Pril", "Shiseido", "Skin1004", "Softlan", "Top",
    "Vim", "Walch"
]

TH_unilever_brands = ["AHC", "Axe", "Baby Dove", "Breeze", "Citra", "Clear", "Comfort", "Dove", "Knorr", "Lifebuoy", "Lux", "Omo", "Pond's", "Rexona", "Sunsilk", "Tresemme", "Vaseline", "Vitakeratin", "Best Foods", "Sunlight"]

TH_unilever_comp1_brands = ["Nivea", "Ariel", "Downy", "Kerastase", "Pantene", "Loreal Paris", "Head & Shoulders", "Bio-Oil", "Neutrogena", "CeraVe", "Dr.Pong", "Skintific", "Aveeno", "Eucerin", "Cetaphil", "Johnson's", "Luxe Organix"]


# ------------------------------
# Use 'page_link' consistently
# ------------------------------
if 'url' in unmapped_df.columns and 'page_link' not in unmapped_df.columns:
    unmapped_df = unmapped_df.rename(columns={'url': 'page_link'})
elif 'url' in unmapped_df.columns and 'page_link' in unmapped_df.columns:
    unmapped_df = unmapped_df.drop(columns=['url'])

# ------------------------------
# Brand lists dictionary
# ------------------------------
brand_lists = {
    "VN": {
        "uni": set([b.lower() for b in VN_unilever_brands]),
        "comp1": set([b.lower() for b in VN_unilever_comp1_brands])
    },
    "MY": {
        "uni": set([b.lower() for b in MY_unilever_brands]),
        "comp1": set([b.lower() for b in MY_unilever_comp1_brands])
    },
    "SG": {
        "uni": set([b.lower() for b in SG_unilever_brands]),
        "comp1": set([b.lower() for b in SG_unilever_comp1_brands])
    },
    "PH": {
        "uni": set([b.lower() for b in PH_unilever_brands]),
        "comp1": set([b.lower() for b in PH_unilever_comp1_brands])
    },
    "TH": {
        "uni": set([b.lower() for b in TH_unilever_brands]),
        "comp1": set([b.lower() for b in TH_unilever_comp1_brands])
    }
}

import re
from difflib import get_close_matches

# Helper: normalize brand strings
def normalize_brand(brand):
    if pd.isna(brand):
        return ""
    brand = str(brand).lower()
    brand = re.sub(r'[^a-z0-9]', '', brand)  # keep only letters/numbers
    brand = brand.strip()
    return brand

# Normalize brand lists once
for country in brand_lists:
    brand_lists[country]['uni'] = set(normalize_brand(b) for b in brand_lists[country]['uni'])
    brand_lists[country]['comp1'] = set(normalize_brand(b) for b in brand_lists[country]['comp1'])

# Precompute normalized predicted brands
unmapped_df['predicted_brand_norm'] = unmapped_df['brandbyModel'].apply(normalize_brand)

# Function to assign uni/comp1 using country_code
def assign_uni_comp1(row):
    brand = row['predicted_brand_norm']
    country = row.get('country_code')  # use the existing country_code column

    # Fallback: if country_code missing, try to infer from URL
    if pd.isna(country) or country == "":
        url = str(row.get('page_link', '')).lower()
        if "lazada.com.my" in url or ".my/" in url or "shopee.my" in url:
            country = "MY"
        elif "lazada.vn" in url or ".vn/" in url or "shopee.vn" in url:
            country = "VN"
        elif "lazada.sg" in url or ".sg/" in url or "shopee.sg" in url:
            country = "SG"
        elif "lazada.ph" in url or ".ph/" in url or "shopee.ph" in url:
            country = "PH"
        elif "lazada.co.th" in url or ".th/" in url or "shopee.co.th" in url:
            country = "TH"
        elif "tiktok" in url:
            country = "VN"  # default or manual mapping for TikTok

    # Assign uni or comp1 based on normalized brand sets
    if country in brand_lists:
        uni_set = brand_lists[country]['uni']
        comp1_set = brand_lists[country]['comp1']

        # Exact match first
        if brand in uni_set:
            return "uni"
        elif brand in comp1_set:
            return "comp1"

        # Fuzzy match as fallback
        if get_close_matches(brand, uni_set, n=1, cutoff=0.8):
            return "uni"
        elif get_close_matches(brand, comp1_set, n=1, cutoff=0.8):
            return "comp1"

    return None

# Apply to all rows
unmapped_df['uni+comp1'] = unmapped_df.apply(assign_uni_comp1, axis=1)



In [26]:
unmapped_df.head(50).T

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
channel_sku_id,1732302988384109654,1733175782840108320,1733306437535827260,1733593556820592398,1731142474860430770,1733295404466407017,1733091627174627308,1730308948275988537,1731911475202393214,1731013029846289249,...,1729668653079103966,1730829738363882301,1729669231404091870,1731074411612441557,1729816375965617067,1729633508483631985,1730139736875698307,1731046429367896824,1730139733839874179,1730856296398291975
price,219.0,284.05,109.0,109.0,195.01,119.0,139.0,69.0,384.12,35.0,...,49.0,199.0,59.0,139.0,45.0,43.0,77.22,26.0,77.22,182.6
mrp,219.0,299.0,398.0,398.0,398.0,398.0,339.0,159.0,776.0,35.0,...,49.0,199.0,59.0,149.0,45.0,43.0,78.0,26.0,78.0,182.6
title,Moisturizing Lightening Lotion Plus 3x Collage...,WHITENING LOTIONS 500ML FROM THAILAND,SIITO Premium Removal - Safe & Natural H...,SIITO Acne Treatments Acne Care Invisible Gel ...,"[QUSTERE Acne Patch] Facial acne patch, colorf...",SIITO Acne Care Invisible Gel is suitable for ...,[NEW LAUNCH] Y.O.U AcnePlus BHA Acne Patch | V...,BUY 1 TAKE 2 YESHUI 40g Green Tea Mud Mask Bla...,【Limited-time offer】QUSTERE facial large hydro...,Mugwort Acne Clay Mask Stick Gentle Cleansing ...,...,Alyssa Luis Adobo Mani NUTS Pack Protein Snack...,Adobo Flakes Classic 220ml,Alyssa Luis MIXED NUTS Pack Protein Snack,MIX Crispy Snacks Peanuts Cornick Popbeans 400g,Mama Sita's Meal Mix Spicy Caldereta 50g,"YM’s Cashew Nuts, Peanut Brittle, Sweet Beans ...",Dingdong Snack Mix Mixed Nuts 95G x 3PCs,"Datu Puti Soy Sauce 350ml - AllMart Grocery, S...",Dingdong Mixed Nuts Original 100G x 3Pcs,"555 Tuna Adobo 155g pack of 6 - Healthy, High ..."
page_link,",https://www.tiktok.com/view/product/173230298...",",https://www.tiktok.com/view/product/173317578...",",https://www.tiktok.com/view/product/173330643...",",https://www.tiktok.com/view/product/173359355...",",https://www.tiktok.com/view/product/173114247...",",https://www.tiktok.com/view/product/173329540...",",https://www.tiktok.com/view/product/173309162...",",https://www.tiktok.com/view/product/173030894...",",https://www.tiktok.com/view/product/173191147...",",https://www.tiktok.com/view/product/173101302...",...,",https://www.tiktok.com/view/product/172966865...",",https://www.tiktok.com/view/product/173082973...",",https://www.tiktok.com/view/product/172966923...",",https://www.tiktok.com/view/product/173107441...",",https://www.tiktok.com/view/product/172981637...",",https://www.tiktok.com/view/product/172963350...",",https://www.tiktok.com/view/product/173013973...",",https://www.tiktok.com/view/product/173104642...",",https://www.tiktok.com/view/product/173013973...",",https://www.tiktok.com/view/product/173085629..."
thumbnail,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-va.ibyteimg.com/tos-maliva-i-o...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...
feat_10,3P,3P,3P,3P,3P,3P,3P,3P,3P,3P,...,3P,3P,3P,3P,3P,3P,3P,3P,3P,3P
country_code,PH,PH,PH,PH,PH,PH,PH,PH,PH,PH,...,PH,PH,PH,PH,PH,PH,PH,PH,PH,PH
title_clean,moisturizing lightening lotion plus 3x collage...,whitening lotions 500ml from thailand,siito premium removal s

**<font size="4">DOWNLOAD DATA</font>**

In [62]:
#download data
#unmapped_df.to_excel('check7.xlsx', index=False, engine='openpyxl')

**<font size="6">STANDARDIZING PACK TYPE</font>**

In [27]:
# Define the mapping dictionary for standardizing pack_type
unit_mapping = {
    'ml': 'ML', 'mL': 'ML', 'ML': 'ML', 'Ml': 'ML',
    'g': 'GM', 'G': 'GM', 'grams': 'GM', 'gram': 'GM',
    'kg': 'KG', 'KG': 'KG', 'kgs': 'KG', 'KGS': 'KG', 'kilo': 'KG', 'KILO': 'KG', 'kilos': 'KG',
    'l': 'L', 'L': 'L', 'liter': 'L', 'liters': 'L', 'litres': 'L',
    'pcs': 'UNIT', 'Pcs': 'UNIT', 'PCS': 'UNIT', 'piece': 'UNIT', 'Pieces': 'UNIT', 'pc': 'UNIT', 'PC': 'UNIT'
}

# Standardize the pack_type values
unmapped_df['pack_type_predicted'] = unmapped_df['pack_type_predicted'].str.lower().map(unit_mapping)
#unmapped_df['pack_type_from_ocr'] = unmapped_df['pack_type_from_ocr'].str.lower().map(unit_mapping)

**<font size="6">QUANTITY COLUMN</font>**

In [28]:
import re

def detect_quantity(title):
    """
    Detects the quantity from a product title.

    Args:
        title (str): The product title.

    Returns:
        int: The detected quantity (default = 1).
    """
    text = title.lower()
    quantity = 1  # Default fallback

    # === PRIORITY 1: If additive volume like "370 ml + 370 ml", treat as 1 unit ===
    additive_match = re.findall(
        r'(\d+(?:\.\d+)?)\s*(ml|g|gm|gr|grms|grams|kg|l|gallon|liter|liters|litre)\s*\+\s*(\d+(?:\.\d+)?)\s*(ml|g|gm|gr|grms|grams|kg|l)',
        text
    )
    if additive_match:
        for m in additive_match:
            unit1 = re.sub(r'\s+', '', m[1])
            unit2 = re.sub(r'\s+', '', m[3])
            if unit1 == unit2:
                return 1

    # === "N x SIZE" e.g. "2 x 62g", "10 x 250ml" ===
    match = re.search(r'(\d+)\s*[x×*]\s*\d+(?:\.\d+)?\s*(ml|g|gm|gr|grams|kg|l|litre|liter|liters)', text)
    if match:
        return int(match.group(1))

    # === Volume/weight followed by "xN" e.g. "490ml x2" ===
    match = re.search(r'\d+(?:\.\d+)?\s*(ml|g|gm|grms|gr|kg|l|grams|litre|liter|liters)\s*[x×]\s*(\d+)', text)
    if match:
        return int(match.group(2))

    # === Promotions (buy n get m, buy n free m, buy n take m) ===
    for pattern in [r'buy\s*(\d+)\s*get\s*(\d+)', r'buy\s*(\d+)\s*free\s*(\d+)', r'buy\s*(\d+)\s*take\s*(\d+)']:
        match = re.search(pattern, text)
        if match:
            return int(match.group(1)) + int(match.group(2))

    # === "N + M" format (but not sizes like 370ml + 370ml) ===
    match = re.search(r'(\d+)\s*\+\s*(\d+)\b(?!\s*(ml|g|kg|l))', text)
    if match:
        return int(match.group(1)) + int(match.group(2))

    # === "xN" format (standalone) ===
    match = re.search(r'\bx\s*(\d+)\b', text)
    if match:
        return int(match.group(1))

    # === Ordered unit keywords: sachets, bags, packs, cups, bottles, etc. ===
    for unit in ['sachet?','sachets?', 'bags?', 'packs?', 'packets?', 'pods?', 'bottles?', 'sticks?', 'tubs?', 'cups?']:
        match = re.search(rf'\b(\d+)\s*{unit}\b', text)
        if match:
            return int(match.group(1))

    # === N-pack, N pack, Npack ===
    match = re.search(r'\b(\d+)\s*[-]?\s*pack\b', text)
    if match:
        return int(match.group(1))

    # === N pieces / pcs / piece ===
    match = re.search(r'\b(\d+)\s*(pieces|pcs|piece)\b', text)
    if match:
        return int(match.group(1))

    # === Dozen = 12 ===
    if "dozen" in text:
        return 12

    # === Keywords like "pack of 3", "set of 4", etc. ===
    keywords = ["bundle of", "pack of", "set of", "combo of", "box of"]
    for keyword in keywords:
        match = re.search(rf'{keyword}\s*(\d+)\b', text)
        if match:
            return int(match.group(1))

    # === Special phrases ===
    special_cases = {
        "pair pack": 2,
        "twin pack": 2,
        "double pack": 2,
        "duo pack": 2,
        "triple pack": 3
    }
    for phrase, qty in special_cases.items():
        if phrase in text:
            return qty

    return quantity


In [29]:
# Apply the function to create a new 'Quantity' column
unmapped_df['QuantitybyModel'] = unmapped_df['title'].apply(detect_quantity)

In [30]:
import pandas as pd
#upload data
#unmapped_df = pd.read_excel('VN_BIDISHA_OLD_QC.xlsx').astype(str)
unmapped_df.head().T

,0,1,2,3,4
channel_sku_id,1732302988384109654,1733175782840108320,1733306437535827260,1733593556820592398,1731142474860430770
price,219.0,284.05,109.0,109.0,195.01
mrp,219.0,299.0,398.0,398.0,398.0
title,Moisturizing Lightening Lotion Plus 3x Collage...,WHITENING LOTIONS 500ML FROM THAILAND,SIITO Premium Removal - Safe & Natural H...,SIITO Acne Treatments Acne Care Invisible Gel ...,"[QUSTERE Acne Patch] Facial acne patch, colorf..."
page_link,",https://www.tiktok.com/view/product/173230298...",",https://www.tiktok.com/view/product/173317578...",",https://www.tiktok.com/view/product/173330643...",",https://www.tiktok.com/view/product/173359355...",",https://www.tiktok.com/view/product/173114247..."
thumbnail,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...
feat_10,3P,3P,3P,3P,3P
country_code,PH,PH,PH,PH,PH
title_clean,moisturizing lightening lotion plus 3x collage...,whitening lotions 500ml from thailand,siito premium removal safe natural hea...,siito acne treatments acne care invisible gel ...,qustere acne patch facial acne patch colorful ...
predicted_category,skin - b&w,irr,irr,irr,irr


**<font size="6">Normalize pack size and pack type of category</font>**

In [31]:
import pandas as pd

# ---------------------------------------------------
# ONE-TIME INPUT (set these only once)
# ---------------------------------------------------
COL_CATEGORY = "predicted_category"   # category column
COL_PACK_TYPE = "pack_size_predicted"              # pack type column
COL_PACK_SIZE = "pack_type_predicted"              # pack size column
# ---------------------------------------------------

def normalize_pack_type_and_size(row):
    category = str(row[COL_CATEGORY]).lower()
    pt = row[COL_PACK_TYPE]
    ps = row[COL_PACK_SIZE]

    if pd.notnull(pt) and pd.notnull(ps):
        try:
            ps = float(ps)
        except ValueError:
            return row   # skip if cannot convert

        # Categories where main unit is LARGE (KG/L)
        if category in ['fabric cleaning - hc', 'fabric enhancer - hc', 'home & hygiene - hc']:

            # GM → KG
            if str(pt).upper() == 'GM':
                row[COL_PACK_TYPE] = 'KG'
                row[COL_PACK_SIZE] = ps / 1000

            # ML → L
            elif str(pt).upper() == 'ML':
                row[COL_PACK_TYPE] = 'L'
                row[COL_PACK_SIZE] = ps / 1000

            # gallon → L
            elif str(pt).lower() == 'gallon':
                row[COL_PACK_TYPE] = 'L'
                row[COL_PACK_SIZE] = ps * 3.785

        else:
            # Reverse logic (KG/L → GM/ML)

            if str(pt).upper() == 'KG':
                row[COL_PACK_TYPE] = 'GM'
                row[COL_PACK_SIZE] = ps * 1000

            elif str(pt).upper() == 'L':
                row[COL_PACK_TYPE] = 'ML'
                row[COL_PACK_SIZE] = ps * 1000

    return row


# ✅ Apply
unmapped_df = unmapped_df.apply(normalize_pack_type_and_size, axis=1)


In [32]:
#Override quantity = 1 where pack_type_predicted == 'UNIT'
unmapped_df.loc[unmapped_df['pack_type_predicted'] == 'UNIT', 'QuantitybyModel'] = 1

**<font size="4">DOWNLOAD DATA</font>**

In [10]:
#download data
#unmapped_df.to_excel('VN_missing_packsize.xlsx', index=False, engine='openpyxl')

**<font size="5">DOWNLOAD DATA</font>**

In [29]:
#unmapped_df.to_excel('VN_test10_model.xlsx', index=False, engine='openpyxl')

**<font size="6">QC</font>**

**<font size="3">IR FLAG QC</font>**

In [125]:
import pandas as pd

# Sample DataFrame
# df = pd.read_csv('your_file.csv')

# Lowercase for consistent matching
#unmapped_df['title_lower'] = unmapped_df['title'].str.lower()
unmapped_df['subcategory_lower'] = unmapped_df['predicted_subcategory'].str.lower()

# General IR keywords from title
general_ir_keywords = [
'make up remover', 'micellar water', 'cleansing water','intimate wash',
    'feminine hygiene','feminine wash','cleansing oil','sanitizer',   
    'body oil',  # body oil relevant only for th, kept here
    'air conditioner','air refresher', 'car spray','car refresher', 
    'shoe spray','shoe refresher','shoes deodorant','shoe deodorant',
    'toothpick','chair','bathtub','exfoliate gloves','exfoliating sponge',
    'stone','exfoliating pads','shower wax','acne patch',
    '[Free Gift]','Free Gift','[NOT FOR SALE]', 'NOT FOR SALE', 
    'sample trial','[HB GIFT]', '[Gift]','Gift','Giftbox', 
    'shirt','womenswear','underwear','comb','laundry basket',
    'pet deodorant','pillow',

    # --- newly added keywords ---
    'trial','sample','not for sale','not for purchase',
    'free gift','bh gift','bw gift','tester','demo pack',
    'complimentary','promotional item','promotional','freebie','bonus pack',
    'display pack','marketing sample','trade sample','promo gift',
    'free trial','gift set'
]

# Subcategory-specific keywords
fabric_cleaning_keywords = ['drum', 'dish', 'washing machine cleaner', 'basket']
shampoo_keywords = ['dog', 'pet']
serum_treatment_keywords = ['dog', 'pet', 'hair removal']

# Define function to apply IR rules
def detect_ir(row):
    title = row['title_clean']
    subcat = row['subcategory_lower']
    
    # Rule 1: General title-based IR match
    if any(keyword in title for keyword in general_ir_keywords):
        return 'IR'
    
    # Rule 2: Subcategory-specific logic
    if subcat == 'fabric cleaning' and any(keyword in title for keyword in fabric_cleaning_keywords):
        return 'IR'
    elif subcat == 'shampoo' and any(keyword in title for keyword in shampoo_keywords):
        return 'IR'
    elif subcat == 'searum & treatment' and any(keyword in title for keyword in serum_treatment_keywords):
        return 'IR'
    
    return ''

# Apply the logic
unmapped_df['IR_flag'] = unmapped_df.apply(detect_ir, axis=1)

# Clean up helper columns
unmapped_df.drop('subcategory_lower', axis=1, inplace=True)


In [32]:
#unmapped_df = pd.read_excel('MODEL_shopee_VN_Bidisha.xlsx')
unmapped_df.head().T

,0,1,2,3,4
channel_sku_id,56703019975,50551972844,26289202872,56403079883,9577124950
title,2000ml Harmony Fragrance Laundry Detergent — S...,Dulce de Leche Yema Spread in can,"Doking Jam 3kg Filling,Spread Orange Blueberry...",Truffle de bola Cheese Spread,Nutella Spread 350 grams
brandbycrawl,0,Alaska,Doking,0,nutella
price,159.0,159.0,285.0,250.0,335.0
max_mrp,300.0,159.0,379.0,250.0,335.0
shop_id,1658030359,570303376,223030186,1014236676,238345847
media,https://down-sg.img.susercontent.com/file/ph-1...,https://down-sg.img.susercontent.com/file/ph-1...,https://down-sg.img.susercontent.com/file/ph-1...,https://down-sg.img.susercontent.com/file/ph-1...,https://down-sg.img.susercontent.com/file/ph-1...
page_link,",https://shopee.ph/2000ml-Harmony-Fragrance-La...",",https://shopee.ph/Dulce-de-Leche-Yema-Spread-...",",https://shopee.ph/Doking-Jam-3kg-Filling,Spre...",",https://shopee.ph/Truffle-de-bola-Cheese-Spre...",",https://shopee.ph/Nutella-Spread-350-grams-i...."
source,category,category,category,category,category
feat_10,3P,3P,3P,3P,3P


**<font size="4">LINKS QC</font>**

In [33]:
import re
import pandas as pd

# -------------------------------------------------
# 1️⃣ Basic URL cleanup
# -------------------------------------------------
def clean_raw_url(url):
    """Basic cleanup: remove commas/spaces and fix protocol typos"""
    if not isinstance(url, str):
        return None

    url = url.strip().lstrip(",").strip()

    # Fix missing colon in protocol
    url = re.sub(r'^https//', 'https://', url)
    url = re.sub(r'^http//', 'http://', url)

    return url


# -------------------------------------------------
# 2️⃣ Enforce https://www.
# -------------------------------------------------
def enforce_https_www(url):
    if not url:
        return None

    # Remove existing protocol
    url = re.sub(r'^https?://', '', url, flags=re.IGNORECASE)

    # Ensure www.
    if not url.startswith("www."):
        url = "www." + url

    return "https://" + url


# -------------------------------------------------
# 3️⃣ Shopee URL fixer
# -------------------------------------------------
def fix_shopee_url(url):
    url = clean_raw_url(url)
    if not url:
        return None

    match = re.search(r'i\.(\d+)\.(\d+)', url)
    if not match:
        return None

    shop_id, item_id = match.groups()

    domain_match = re.search(r'shopee\.[a-z\.]+', url)
    domain = domain_match.group(0) if domain_match else "shopee.com"

    fixed_url = f"https://www.{domain}/product-i.{shop_id}.{item_id}"
    return fixed_url


# -------------------------------------------------
# 4️⃣ Lazada URL fixer
# -------------------------------------------------
def fix_lazada_url(url):
    url = clean_raw_url(url)
    if not url:
        return None

    # Add protocol if missing
    if not re.match(r'^https?://', url):
        url = "https://" + url.lstrip("/")

    # Force www
    url = re.sub(r'^https?://lazada\.', 'https://www.lazada.', url)

    match = re.search(
        r'https://www\.lazada\.[a-z\.]+/products/[^\s]+-i\d+\.html',
        url
    )

    if match:
        return match.group(0)

    return enforce_https_www(url) if "lazada" in url.lower() else None


# -------------------------------------------------
# 5️⃣ TikTok URL fixer
# -------------------------------------------------
def fix_tiktok_url(url):
    url = clean_raw_url(url)
    if not url:
        return None

    return enforce_https_www(url) if "tiktok" in url.lower() else None


# -------------------------------------------------
# 6️⃣ Main router
# -------------------------------------------------
def fix_url(row):
    url = row.get("page_link") or row.get("url")

    if not isinstance(url, str) or not url.strip():
        return None

    url_lower = url.lower()

    if "shopee" in url_lower:
        return fix_shopee_url(url)
    elif "lazada" in url_lower:
        return fix_lazada_url(url)
    elif "tiktok" in url_lower:
        return fix_tiktok_url(url)

    # Default: still enforce https://www.
    return enforce_https_www(clean_raw_url(url))


# -------------------------------------------------
# 7️⃣ Apply to DataFrame
# -------------------------------------------------
unmapped_df["valid_url"] = unmapped_df.apply(fix_url, axis=1)


In [34]:
unmapped_df.head().T

,0,1,2,3,4
channel_sku_id,1732302988384109654,1733175782840108320,1733306437535827260,1733593556820592398,1731142474860430770
price,219.0,284.05,109.0,109.0,195.01
mrp,219.0,299.0,398.0,398.0,398.0
title,Moisturizing Lightening Lotion Plus 3x Collage...,WHITENING LOTIONS 500ML FROM THAILAND,SIITO Premium Removal - Safe & Natural H...,SIITO Acne Treatments Acne Care Invisible Gel ...,"[QUSTERE Acne Patch] Facial acne patch, colorf..."
page_link,",https://www.tiktok.com/view/product/173230298...",",https://www.tiktok.com/view/product/173317578...",",https://www.tiktok.com/view/product/173330643...",",https://www.tiktok.com/view/product/173359355...",",https://www.tiktok.com/view/product/173114247..."
thumbnail,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...,https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...
feat_10,3P,3P,3P,3P,3P
country_code,PH,PH,PH,PH,PH
title_clean,moisturizing lightening lotion plus 3x collage...,whitening lotions 500ml from thailand,siito premium removal safe natural hea...,siito acne treatments acne care invisible gel ...,qustere acne patch facial acne patch colorful ...
predicted_category,skin - b&w,irr,irr,irr,irr


**<font size="6">PARENT ID</font>**

In [ ]:
unmapped_df = unmapped_df.copy()
# -------------------------------------------------
# 1️⃣ Identify link column (page_link OR url)
# -------------------------------------------------
link_col = None
for c in ["page_link", "url", "valid_link"]:
    if c in unmapped_df.columns:
        link_col = c
        break
if link_col is None:
    raise ValueError("No page_link or url column found")
# -------------------------------------------------
# 2️⃣ Filter Lazada rows only
# -------------------------------------------------
lazada_mask = unmapped_df[link_col].astype(str).str.contains("lazada", case=False, na=False)
# -------------------------------------------------
# 3️⃣ Create parent_id
# -------------------------------------------------
unmapped_df.loc[lazada_mask, "parent_id"] = (
    unmapped_df.loc[lazada_mask, "channel_sku_id"]
    .astype(str)
    .str.split("-")
    .str[0]
)
# -------------------------------------------------
# 4️⃣ Priority selection helpers
# -------------------------------------------------
def pick_non_irr(series):
    s = series.dropna().astype(str).str.strip().str.lower()
    non_irr = s[s != "irr"]
    return non_irr.iloc[0] if not non_irr.empty else s.iloc[0] if not s.empty else None
def pick_real_brand(series):
    s = series.dropna().astype(str).str.strip()
    bad = ["other", "unknown", ""]
    good = s[~s.str.lower().isin(bad)]
    return good.iloc[0] if not good.empty else s.iloc[0] if not s.empty else None
# -------------------------------------------------
# 5️⃣ Harmonise values at parent_id level (Lazada only)
# -------------------------------------------------
group_cols = ["predicted_category", "predicted_subcategory", "brandbyModel"]

for col in group_cols:
    if col not in unmapped_df.columns:
        continue
    if col == "brandbyModel":
        chosen = (
            unmapped_df[lazada_mask]
            .groupby("parent_id")[col]
            .apply(pick_real_brand)
        )
    else:
        chosen = (
            unmapped_df[lazada_mask]
            .groupby("parent_id")[col]
            .apply(pick_non_irr)
        )
    unmapped_df.loc[lazada_mask, col] = (
        unmapped_df.loc[lazada_mask, "parent_id"]
        .map(chosen)
    )
# -------------------------------------------------
# 6️⃣ Reorder columns
# -------------------------------------------------
cols = unmapped_df.columns.tolist()
new_order = (
    ["channel_sku_id", "parent_id"]
    + [c for c in cols if c not in ["channel_sku_id", "parent_id"]]
)
unmapped_df = unmapped_df[new_order]


**<font size="5">check other brand</font>**

In [ ]:
brand_master = pd.read_excel("Brand_list_all_country.xlsx")
brand_master["brand"] = brand_master["brand"].str.lower().str.strip()
brand_master["country_code"] = brand_master["country_code"].str.upper().str.strip()
# ❌ Exclude generic brands from other_brand logic
exclude_brands = {"comfort", "all", "hygiene", "simple", "breeze"}
brand_master = brand_master[~brand_master["brand"].isin(exclude_brands)]
# -------------------------------
# Normalize unmapped_df
# -------------------------------
unmapped_df["sku_title_norm"] = unmapped_df["title"].str.lower()
unmapped_df["predicted_brand"] = unmapped_df["brandbyModel"].astype(str).str.strip()
unmapped_df["country_code"] = unmapped_df["country_code"].str.upper().str.strip()
# Create other_brand column (default blank)
unmapped_df["other_brand"] = ""
# -------------------------------
# Country-wise regex matching
# -------------------------------
for country in brand_master["country_code"].unique():
    brands = (
        brand_master.loc[
            brand_master["country_code"] == country, "brand"
        ]
        .dropna()
        .sort_values(key=lambda x: x.str.len(), ascending=False)  # longest first
        .unique()
    )
    if len(brands) == 0:
        continue
    # 🔒 STRICT whole-word match (no substrings)
    # Matches only if brand is surrounded by start/end, space, or punctuation
    pattern = r"(?<!\w)(" + "|".join(map(re.escape, brands)) + r")(?!\w)"
    regex = re.compile(pattern)

    mask = (
        (unmapped_df["country_code"] == country) &
        (
            unmapped_df["predicted_brand"].isna() |
            (unmapped_df["predicted_brand"] == "") |
            (unmapped_df["predicted_brand"].str.lower() == "nan")
        )
    )
    unmapped_df.loc[mask, "other_brand"] = (
        unmapped_df.loc[mask, "sku_title_norm"]
        .str.extract(regex, expand=False)
        .fillna("")
    )
# -------------------------------
# Cleanup
# -------------------------------
unmapped_df.drop(columns=["sku_title_norm"], inplace=True)
unmapped_df["other_brand"] = unmapped_df["other_brand"].str.title()


**<font size="5">Shudhanshu_skus</font>**

In [ ]:
shudhanshu_file = "ph_shudhanshu.xlsx"
# -------------------------------
# Load Shudhanshu SKUs
# -------------------------------
shudhanshu_df = pd.read_excel(shudhanshu_file)
shudhanshu_df['channel_sku_id'] = (
    shudhanshu_df['channel_sku_id']
    .astype(str)
    .str.strip()
)
# Optional: normalize country if you want to use later
shudhanshu_df['country_code'] = (
    shudhanshu_df['country_code']
    .astype(str)
    .str.upper()
    .str.strip()
)
# -------------------------------
# Normalize unmapped_df SKU
# -------------------------------
unmapped_df['channel_sku_id'] = (
    unmapped_df['channel_sku_id']
    .astype(str)
    .str.strip()
)
# -------------------------------
# Create match flag
# -------------------------------
shudhanshu_sku_set = set(shudhanshu_df['channel_sku_id'])
unmapped_df['shudhanshu_skus'] = unmapped_df['channel_sku_id'].isin(shudhanshu_sku_set)


C:\Users\bidis\AppData\Local\Programs\Python\Python314\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**<font size="5">DOWNLOAD DATA</font>**

In [38]:
unmapped_df.to_excel('PH_tiktok_jan_feb18_Model.xlsx', index=False, engine='openpyxl')

In [171]:
unmapped_df.head(200).T

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
channel_sku_id,2546839177-s9058034703,310078297-s550176881,5173035514-s21927398567,5909933756-s25283898692,5394446284-s22917457712,5889596920-s25155624314,5709675386-s24348696326,5646851270-s24062015326,5140342211-s21717067556,5058580829-s21382128973,...,5144772937-s21733960698,4554481896-s18543266784,5863759945-s24985707356,5799087725-s24695903626,5365112484-s23224814092,5809756130-s24738762158,5876485273-s25041931293,5149518656-s23089166919,5781845018-s24611972057,3846643725-s25328286473
parent_id,2546839177,310078297,5173035514,5909933756,5394446284,5889596920,5709675386,5646851270,5140342211,5058580829,...,5144772937,4554481896,5863759945,5799087725,5365112484,5809756130,5876485273,5149518656,5781845018,3846643725
channel_id,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,31,31,31,31,31,31
title,"Instantly Ageless (sold individually, 1 vial) ...",Instantly Ageless eye cream for reducing under...,100% Genuine [Exp.05/2027] Instantly Ageless J...,Peter Thomas Roth Instant Firmx Easy Wear 20ml...,Ultrasun Eye Protection SPF30: Sunscreen and e...,Sadoer Collagen Anti-Aging Eye Cream Eye Care ...,Luna Jelly Eye Care Cream with Massage Head 10...,"Tomato Powerful Serum (under-eye gel), 20 gram...",🔥Fast results in 1 day!🔥 West&Month Eyelash Gr...,"【Buy 1 Get 1 Free】Aichun Eye Cream, Under-Eye ...",...,This laundry detergent making kit makes 15 lit...,Essence Magic Wash 700 ml. Detergent,(2 แถม 2) Kleenit Concentrated Laundry Deterge...,"Laundry detergent, 1 gallon (4500 ml), delicat...","Fineline Concentrated Laundry Detergent, 30 ml...",🔥16 bags - 8000ml🔥 Rose Scented 4-in-1 Laundry...,(Buy 4 Get 4 Free) Hiwny Fabric Softener + Fab...,"SURE fabric softener, Lily Romance scent, 500 ml.",Hygene extra concentrated fabric softener 480 ...,Clean D Fabric Softener..Purple..Violet Elegan...
price,189.0,800.0,179.0,863.0,1190.0,23.7,159.0,150.0,129.0,129.0,...,189.0,99.0,80.0,110.0,55.0,239.0,230.0,14.0,55.0,44.0
mrp,299.0,-1.0,-1.0,1598.0,1590.0,99.0,259.0,190.0,300.0,387.0,...,195.0,-1.0,-1.0,500.0,-1.0,600.0,-1.0,15.0,79.0,45.0
source,category,category,category,category,category,category,category,category,category,category,...,category,category,category,category,category,category,category,category,category,category
sellerId,100108502,100190149,100109916,101304864615,100245654261,101280400986,100877808755,100190175588,100240395034,100331568057,...,100392400075,100179981567,100541104159,100578768083,100711504151,101217920142,101304368145,1000310588,100201215220,100212756804
brandbycrawl,No Brand,Jeunesse,Instantly Ageless,No Brand,Ultrasun,No Brand,Luna,Tomato,No Brand,No Brand,...,No Brand,Essence,No Brand,No Brand,Fineline,No Brand,No Brand,Sure,Hygiene,Cleandee (คลีนดี)
thumbnail,https://filebroker-cdn.lazada.co.th/kf/S50854e...,https://filebroker-cdn.lazada.co.th/kf/S0f3f13...,https://filebroker-cdn.lazada.co.th/kf/S78121d...,https://th-live-01.slatic.net/p/cb3f323c51b457...,https://th-live.slatic.net/p/2d74114e991a87be3...,https://th-live-01.slatic.net/p/b63474be8ee38c...,https://th-live-01.slatic.net/p/633e7c46e657ca...,https://th-live-01.slatic.net/p/ba785b51f51eed...,https://th-live-01.slatic.net/p/7fb7686cb7a2e0...,https://th-live-01.slatic.net/p/55dfd9acd00ceb...,...,https://th-live.slatic.net/p/54cf6edf94f129e8f...,https://th-live-01.slatic.net/p/4d4adab764ed44...,https://sg-test-11.slatic.net/p/ed3528a940a996...,https://sg-test-11.slatic.net/p/ee3260280bbf13...,https://filebroker-cdn.lazada.co.th/kf/Seb3b63...,https://sg-test-11.slatic.net/p/70f10645489a8f...,https://sg-test-11.slatic.net/p/a3ffb9e459ff13...,https://filebroker-cdn.lazada.co.th/kf/S586a26...,https://th-live.slatic.net/p/21a27efaac36dd852...,https://th-live-01.slatic.net/p/881683fcb5a3cb...


**<font size="6">DOWNLOAD QC DATA</font>**


In [4]:
unmapped_df.to_excel('SG_LAZADA_AUGUST_SEP_OCT5_model_normalized.xlsx', index=False, engine='openpyxl')

**<font size="5">SKIP THIS PART*********************************************************</font>**

In [ ]:
#*****************************************pack_size for screenshot********************************************
#****************************************upload the image path only from system***************************************
from PIL import Image
from pytesseract import pytesseract

# Defining paths to tesseract.exe
# and the image we would be using
path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
image_path = r"C:\Users\hp\Pictures\Screenshots\Screenshot 2025-06-13 112616.png"

# Opening the image & storing it in an image object
img = Image.open(image_path)

# Providing the tesseract executable
# location to pytesseract library
pytesseract.tesseract_cmd = path_to_tesseract

# Passing the image object to image_to_string() function
# This function will extract the text from the image
text = pytesseract.image_to_string(img)

# Displaying the extracted text
print(text[:-1])

In [ ]:
#*****************************************pack_size for thumbnail link********************************************
#****************************************upload the thumbnail like only***************************************
import requests
import io
from PIL import Image, ImageEnhance, ImageFilter
from pytesseract import pytesseract
import re

# Setup path to tesseract executable
pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Thumbnail image URL
img_url = "https://p16-oec-sg.ibyteimg.com/tos-alisg-i-aphluv4xwc-sg/fd3c773156cf49f894b258182eb8c2ce~tplv-aphluv4xwc-resize-webp:400:400.webp?dr=15582&from=1351027591&idc=my2&ps=933b5bde&shcp=9b759fb9&shp=50870b87&t=555f072d"
# Step 1: Download image
response = requests.get(img_url, headers={"User-Agent": "Mozilla/5.0"})
image = Image.open(io.BytesIO(response.content))

# Step 2: Preprocess image (basic grayscale + sharpness + contrast)
gray = image.convert("L")  # convert to grayscale
enhancer = ImageEnhance.Contrast(gray).enhance(2.0)  # improve contrast
sharp = enhancer.filter(ImageFilter.SHARPEN)  # sharpen text

# Step 3: OCR
text = pytesseract.image_to_string(sharp)
print("📝 OCR Text:\n", text)

# Step 4: Extract pack size and type using regex
matches = re.findall(r'(\d+(?:[.,]\d+)?)\s*(ml|g|kg|l|litre|liter|gram)', text.lower())
if matches:
    size, unit = matches[0]
    unit = {'gram': 'g', 'liter': 'l', 'litre': 'l'}.get(unit, unit)
    print(f"📦 Pack Size: {size}, Pack Type: {unit}")
else:
    print("📦 No pack size/type detected.")


**<font size="6">PACK SIZE WITH IMAGE LINKS</font>**

In [87]:
import re

import pandas as pd
#upload data
#unmapped_df = pd.read_excel('VN_Lazada_newAsins_Aug_By_Model_forPACKBANDS.xlsx').astype(str)

In [ ]:
import re
import pandas as pd

def extract_pack_size_and_type(title):
    text = title.lower()
    text = text.replace(",", "").replace("×", "x").replace("&", "and")

    # --- remove promotions like "buy 1 get 2 free", "b1g1", "buy 2 get 3"
    text = re.sub(r"buy\s*\d+\s*get\s*\d+(\s*free)?", "", text)
    text = re.sub(r"\bb\s*\d+\s*g\s*\d+\b", "", text)

    # --- remove brand-like tokens with letters+numbers together (e.g. glad2glow, g2g, b2b)
    text = re.sub(r"\b[a-z]+\d+[a-z]+\b", "", text)

    # --- remove multipack mentions not sizes (3pcs, 10 pieces, etc.)
    text = re.sub(r"\b\d+\s*(pcs|pieces|piece|pack)\b", "", text)

    unit_map = {
        'g': 'g', 'gm': 'g', 'gr': 'g', 'grams': 'g',
        'kg': 'kg', 'kgs': 'kg', 'kilo': 'kg',
        'ml': 'ml', 'l': 'l', 'liter': 'l', 'liters': 'l', 'litre': 'l',
        'gallon': 'gallon'
    }

    # --- 1. Range pattern "10-16l"
    match = re.search(r'(\d+(?:\.\d+)?)[-–](\d+(?:\.\d+)?)(\s*ml|\s*g|\s*gm|\s*gr|\s*grams|\s*kg|\s*l|\s*gallon|\s*liter|\s*liters|\s*litre|\s*kilo)', text)
    if match:
        size = min(float(match.group(1)), float(match.group(2)))
        unit = unit_map.get(match.group(3).strip(), match.group(3).strip())
        if unit == "kg": size *= 1000; unit = "g"
        elif unit == "l": size *= 1000; unit = "ml"
        return size, unit

    # --- 2. Slash-separated values "1kg/500g/250g"
    match = re.findall(r'(\d+(?:\.\d+)?)(kg|g|gm|gr|grams|ml|l)', text)
    if "/" in text and match:
        values = []
        final_unit = None
        for val, unit in match:
            size = float(val)
            unit = unit_map.get(unit.strip(), unit.strip())
            if unit == "kg": size *= 1000; unit = "g"
            elif unit == "l": size *= 1000; unit = "ml"
            values.append(size)
            final_unit = unit
        return min(values), final_unit

    # --- 3. Standard matches (handles parentheses/brackets)
    match = re.findall(r'(\d+(?:\.\d+)?)(\s*ml|\s*g|\s*gm|\s*gr|\s*grams|\s*kg|\s*kgs|\s*l|\s*gallon|\s*liter|\s*liters|\s*litre|\s*kilo)', text)
    if match:
        sizes = []
        final_unit = None
        for val, unit in match:
            size = float(val)
            unit = unit_map.get(unit.strip(), unit.strip())
            if unit == "kg": size *= 1000; unit = "g"
            elif unit == "l": size *= 1000; unit = "ml"
            sizes.append(size)
            final_unit = unit

        if sizes:
            if ("+" in text) or (" and " in text):
                return sum(sizes), final_unit
            else:
                return min(sizes), final_unit

    return None, None


In [ ]:
import pandas as pd
import requests
import io
from PIL import Image, ImageEnhance, ImageFilter
from pytesseract import pytesseract as PT
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import MarianMTModel, MarianTokenizer

# --- Tesseract path ---
PT.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# --- Load MarianMT offline translation model ---
model_name = "Helsinki-NLP/opus-mt-mul-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# --- Translation function ---
def translate_to_english(text):
    if not isinstance(text, str) or not text.strip():
        return text
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# --- Extract image from URL ---
def get_image(url):
    try:
        if not isinstance(url, str) or not url.strip():
            return None
        resp = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        if resp.status_code != 200:
            return None
        image = Image.open(io.BytesIO(resp.content)).convert("L")
        return image
    except Exception:
        return None

# --- OCR function ---
def ocr_image(img):
    if img is None:
        return None
    try:
        enhancer = ImageEnhance.Contrast(img).enhance(2.0)
        sharp = enhancer.filter(ImageFilter.SHARPEN)
        text = PT.image_to_string(sharp)
        return text
    except Exception:
        return None

# --- Process a single row ---
def process_row(row):
    thumbnail_url = row.get('thumbnail')
    if not thumbnail_url or not isinstance(thumbnail_url, str):
        return pd.Series([None, None, None, None])
    
    # 1️⃣ OCR
    img = get_image(thumbnail_url)
    ocr_text = ocr_image(img)
    
    # 2️⃣ Translate
    ocr_text_translated = translate_to_english(ocr_text) if ocr_text else None
    
    # 3️⃣ Extract pack size and unit using existing function
    if ocr_text_translated:
        pack_size, pack_type = extract_pack_size_and_type(ocr_text_translated)
    else:
        pack_size, pack_type = None, None
    
    return pd.Series([ocr_text, ocr_text_translated, pack_size, pack_type])

# --- Prepare output columns ---
#unmapped_df['ocr_text'] = None
#unmapped_df['ocr_translated_text'] = None
unmapped_df['pack_size_from_ocr'] = None
unmapped_df['pack_type_from_ocr'] = None

# --- Multithreaded processing ---
def apply_ocr_translate(df, max_workers=8):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_row, row): idx for idx, row in df.iterrows()}
        for future in tqdm(as_completed(futures), total=len(futures), desc="🔍 OCR + Translate + Pack Size"):
            idx = futures[future]
            try:
                ocr_text, ocr_text_translated, pack_size, pack_type = future.result()
                #df.at[idx, 'ocr_text'] = ocr_text
                #df.at[idx, 'ocr_translated_text'] = ocr_text_translated
                df.at[idx, 'pack_size_from_ocr'] = pack_size
                df.at[idx, 'pack_type_from_ocr'] = pack_type
            except Exception:
                #df.at[idx, 'ocr_text'] = None
                #df.at[idx, 'ocr_translated_text'] = None
                df.at[idx, 'pack_size_from_ocr'] = None
                df.at[idx, 'pack_type_from_ocr'] = None
    return df

# --- Run the pipeline ---

# --- Filter rows based on uni+comp1 and predicted_category ---
mask = (unmapped_df['uni+comp1'].isin(['uni', 'comp1'])) & (unmapped_df['predicted_category'] != 'irr') &(unmapped_df['pack_size_predicted'] == "")
df_to_process = unmapped_df[mask].copy()

# --- Multithreaded OCR + Translate + Pack Size ---
df_processed = apply_ocr_translate(df_to_process, max_workers=8)

# --- Update only the relevant rows in the original DataFrame ---
unmapped_df.loc[mask, 'pack_size_from_ocr'] = df_processed['pack_size_from_ocr'].values
unmapped_df.loc[mask, 'pack_type_from_ocr'] = df_processed['pack_type_from_ocr'].values
#unmapped_df = apply_ocr_translate(unmapped_df, max_workers=8)


#clean text before saving
for col in ['ocr_translated_text', 'ocr_text']:
    if col in unmapped_df.columns:
        unmapped_df[col] = unmapped_df[col].astype(str).apply(lambda x: "'" + x if x.startswith('=') else x)

for col in ['ocr_text', 'ocr_translated_text']:
    if col in unmapped_df.columns:
        unmapped_df[col] = unmapped_df[col].astype(str).str.replace(r'[\x00-\x1F]+', ' ', regex=True)

max_len = 32000  # Excel limit for a cell
for col in ['ocr_text', 'ocr_translated_text']:
    if col in unmapped_df.columns:
        unmapped_df[col] = unmapped_df[col].str.slice(0, max_len)


In [ ]:
# Define the mapping dictionary for standardizing pack_type
unit_mapping = {
    'ml': 'ML', 'mL': 'ML', 'ML': 'ML', 'Ml': 'ML',
    'g': 'GM', 'G': 'GM', 'grams': 'GM', 'gram': 'GM',
    'kg': 'KG', 'KG': 'KG', 'kgs': 'KG', 'KGS': 'KG', 'kilo': 'KG', 'KILO': 'KG', 'kilos': 'KG',
    'l': 'L', 'L': 'L', 'liter': 'L', 'liters': 'L', 'litres': 'L',
    'pcs': 'UNIT', 'Pcs': 'UNIT', 'PCS': 'UNIT', 'piece': 'UNIT', 'Pieces': 'UNIT', 'pc': 'UNIT', 'PC': 'UNIT',
    'gallon': 'GALLON', 'Gallon': 'GALLON', 'GALLON': 'GALLON'
}

# Standardize the pack_type values
unmapped_df['pack_type_predicted'] = unmapped_df['pack_type_predicted'].str.lower().map(unit_mapping)
#unmapped_df['pack_type_from_ocr'] = unmapped_df['pack_type_from_ocr'].str.lower().map(unit_mapping)